In [1]:
import tensorflow as tf
import datetime
import os

import numpy as np
import matplotlib.pyplot as plt

2023-05-26 17:13:23.396095: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Prepare Datasets

 * Load train dataset and split into train and validation sets
 * Load test dataset

In [2]:
train, validation = tf.keras.utils.image_dataset_from_directory(
    '../Dataset/train',
    labels='inferred',
    label_mode='categorical',
    image_size=(224,224),
    seed=815,
    validation_split=0.1,
    subset='both'
)

test = tf.keras.utils.image_dataset_from_directory(
    '../Dataset/test',
    labels='inferred',
    label_mode='categorical',
    image_size=(224,224)
)

Found 22573 files belonging to 25 classes.
Using 20316 files for training.
Using 2257 files for validation.
Found 2500 files belonging to 25 classes.


2023-05-26 17:13:25.378908: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-26 17:13:25.382356: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-26 17:13:25.382515: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
if not os.path.isdir('models'):
    os.mkdir('models')

### Create model
 * Convolutional block
   * Residual "skipping" layer
   * Batch normalization around each convolutional layer
   * (2,2) Max Pooling with stride (2,2) after all convulutional layers
 * Flatten
 * Dense layers with ReLu activation
 * Output layer with Softmax activation
 * Model compiled with adam optimizer

In [4]:
def generate_model(conv_layers, dense_layers):    
    img_input = tf.keras.layers.Input(shape=(224, 224, 3))
    x = tf.keras.layers.BatchNormalization()(img_input)
    
    for filters, kernel_size, depth, pool in conv_layers:
        shortcut = tf.keras.layers.Conv2D(filters, 1, activation='relu')(x)
        shortcut = tf.keras.layers.BatchNormalization()(shortcut)
        
        for i in range(depth):
            x = tf.keras.layers.Conv2D(                
                filters,
                kernel_size,
                activation='relu',
                padding='same'
            )(x)
            x = tf.keras.layers.BatchNormalization()(x)
                
        x = tf.keras.layers.Add()([shortcut,x])
        x = tf.keras.layers.Activation('relu')(x)
            
        if pool > 1:
            x = tf.keras.layers.MaxPool2D(
                pool_size=pool
            )(x)
            
        x = tf.keras.layers.BatchNormalization()(x)
        
    x = tf.keras.layers.Flatten()(x)
    
    for units in dense_layers:
        x = tf.keras.layers.Dense(
            units,
            activation='relu'
        )(x)
    
    x = tf.keras.layers.Dense(
        25,
        activation='softmax'
    )(x)
    
    
    model = tf.keras.Model(img_input, x)
    
    model.compile(
        optimizer='adam',
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=['accuracy']
    )
    
    return model

### Run 

 * Instantiate model with set parameters
 * Save model json to file
 * Fit model to training dataset
   * Early stopping callback
   * TensorBoard callback
   * Model checkpoint callback
 * Evaluate model with test dataset

In [5]:
def run(id):
    model = generate_model(
        [
            # filters, kernel_size, depth, pool
            (32, 3, 2, 2),
            (64, 3, 2, 2),
            (128, 3, 2, 2),
        ], 
        [
            # units
            128,
            192,
        ])
    model.summary()
    
    with open(f'models/{id}.json', 'w') as f:
        f.write(model.to_json())
        
    model.fit(
        train,
        validation_data=validation,
        epochs=25, 
        #batch_size=16,
        callbacks=[
            tf.keras.callbacks.EarlyStopping(),
            tf.keras.callbacks.TensorBoard(
                log_dir=f'logs/fit/{id}'
            ),
            tf.keras.callbacks.ModelCheckpoint(
                filepath=f'checkpoints/{id}',
                monitor='val_accuracy',
            )
        ]
    )
        
    model.evaluate(
        test, 
        callbacks=[
            tf.keras.callbacks.TensorBoard(
                log_dir=f'logs/evaluate/{id}'
            )
        ]
    )
    

now = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
for i in range(10):
    run(f'{now}-{i}')

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 batch_normalization (BatchNorm  (None, 224, 224, 3)  12         ['input_1[0][0]']                
 alization)                                                                                       
                                                                                                  
 conv2d_1 (Conv2D)              (None, 224, 224, 32  896         ['batch_normalization[0][0]']    
                                )                                                             

 add_2 (Add)                    (None, 56, 56, 128)  0           ['batch_normalization_9[0][0]',  
                                                                  'batch_normalization_11[0][0]'] 
                                                                                                  
 activation_2 (Activation)      (None, 56, 56, 128)  0           ['add_2[0][0]']                  
                                                                                                  
 max_pooling2d_2 (MaxPooling2D)  (None, 28, 28, 128)  0          ['activation_2[0][0]']           
                                                                                                  
 batch_normalization_12 (BatchN  (None, 28, 28, 128)  512        ['max_pooling2d_2[0][0]']        
 ormalization)                                                                                    
                                                                                                  
 flatten (

2023-05-26 17:13:26.388087: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [20316]
	 [[{{node Placeholder/_4}}]]
2023-05-26 17:13:26.388333: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [20316]
	 [[{{node Placeholder/_4}}]]
2023-05-26 17:13:28.499157: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-05-26 17:13:30.186947: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f12d0062ed0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2

635/635 [==============================] - ETA: 0s - loss: 3.2732 - accuracy: 0.2349

2023-05-26 17:15:21.509727: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2257]
	 [[{{node Placeholder/_4}}]]
2023-05-26 17:15:21.509896: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2257]
	 [[{{node Placeholder/_4}}]]


...

INFO:tensorflow:Assets written to: checkpoints/20230526-171326-9/assets


635/635 [==============================] - 109s 172ms/step - loss: 0.9224 - accuracy: 0.7156 - val_loss: 1.7638 - val_accuracy: 0.5419
Epoch 6/25
635/635 [==============================] - ETA: 0s - loss: 0.7078 - accuracy: 0.7819

INFO:tensorflow:Assets written to: checkpoints/20230526-171326-9/assets


INFO:tensorflow:Assets written to: checkpoints/20230526-171326-9/assets


79/79 [==============================] - 4s 45ms/step - loss: 1.9996 - accuracy: 0.5336
